# Useful links

- [ ] https://cran.r-project.org/web/packages/afex/vignettes/introduction-mixed-models.pdf
- [ ] https://cran.r-project.org/web/packages/afex/afex.pdf
- [ ] https://cran.r-project.org/web/packages/afex/index.html
- [ ] https://cran.r-project.org/web/packages/afex/vignettes/afex_analysing_accuracy_data.html
- [ ] https://cran.r-project.org/web/packages/afex/vignettes/afex_anova_example.html
- [ ] https://cran.r-project.org/web/packages/afex/vignettes/afex_mixed_example.html
- [ ] https://cran.r-project.org/web/packages/emmeans/vignettes/comparisons.html
- [ ] https://cran.r-project.org/web/packages/emmeans/vignettes/messy-data.html
- [ ] https://cran.r-project.org/web/packages/emmeans/vignettes/interactions.html
- [ ] https://cran.r-project.org/web/packages/emmeans/vignettes/basics.html
- [ ] https://cran.r-project.org/web/packages/afex/vignettes/afex_mixed_example.html

- [ ] https://osf.io/nfwx7/

- [ ] https://stats.stackexchange.com/questions/13166/rs-lmer-cheat-sheet

# Setup

## Imports

In [16]:
library("dplyr")
library("tidyr")
library("afex")     # provides mixed() and attaches lme4 automatically.
library("emmeans")  # follow-up tests.
library("multcomp") # advanced control for multiple testing/Type 1 errors.
library("ggplot2")

In [17]:
library("car")
require("lattice")
require("ez")

## Settings

In [18]:
theme_set(theme_bw(base_size = 15) +
            theme(legend.position="bottom",
                  panel.grid.major.x = element_blank()))

In [19]:
options(width=120)

# Create output directory

In [20]:
dir.create("data/t1007",showWarnings = FALSE,recursive = TRUE)

## Handy functions

### Function for logging information BEFORE execution

In [21]:
logging0 <- function(ifName,infoStr=""){
    t0 <- Sys.time()
    s0 <- gsub("\\s","_",t0)
    s1 <- gsub("^.","",tempfile("","",""))
    s2 <- gsub("/t1006/","/t1007/",ifName)
    ofName <- paste(paste(s2,"",s0,"",s1,infoStr,"_nrow",nrow(df3),sep="_"),".RData",sep="")
    lfName = paste(ofName ,".LOG.txt" ,sep="" ,collapse = "")
    log_msg <- toString(paste(
        paste("logs0" ,toString(lfName)     ,sep=": " ,collapse = "")
      , paste("time0" ,toString(Sys.time()) ,sep=": " ,collapse = "")
      , paste("nrows" ,toString(nrow(df3))  ,sep=": " ,collapse = "")
      , paste("ncols" ,toString(ncol(df3))  ,sep=": " ,collapse = "")
      , paste("vList" ,toString(ls())       ,sep=": " ,collapse = "")
      , paste("oSave" ,toString(ofName)     ,sep=": " ,collapse = "")        
      , "\n"
      , sep="\n"))
    cat(log_msg)
    write(paste(log_msg) ,file=lfName ,append=TRUE)
    return(list("ofName"=ofName,"lfName"=lfName,"t0"=t0))
}

### Function for logging information AFTER execution

In [22]:
logging1 <- function(lfName,t0){
    t1 <- Sys.time()
    d1 <- t1-t0
    log_msg <- toString(paste(
        paste("time1" ,toString(Sys.time()) ,sep=": " ,collapse = "")
      , paste("diff1" ,capture.output(d1)   ,sep=": " ,collapse = "")
      , "\n"
      , sep="\n"))
    cat(log_msg)
    write(paste(log_msg) ,file=lfName ,append=TRUE)
    return(list("d1" = d1))
}

# Load data

Depending on how much data you want to include select a file to be loaded (either by executing relevant cell or by clicking ↑ or ↓ above to move cells up or down, the last executed cell containing `ifName` defines the file to be loaded).

In [23]:
ifName="data/t1006/bigDF__df1.RData" # 752554 obs

In [24]:
load(file=ifName)

## Select less

In [26]:
df2 <- select(
    df1
  , LBL
  , wordCnt
  , vb01Cnt
  , nn01Cnt
  , ag01Cnt
  , ag02Cnt
  , vb01Prc
  , nn01Prc
  , ag01Prc
  , ag02Prc
  , ID
)

ERROR: Error in select(df1, LBL): unused argument (LBL)


## Split label

# Quick look at data

In [15]:
str(df1)

'data.frame':	752554 obs. of  28 variables:
 $ LBL              : Factor w/ 4 levels "metoo_ctrl","metoo_orig",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ wordCnt          : num  11 7 11 27 30 26 8 12 12 23 ...
 $ vb01Cnt          : num  0 0 0 2 0 0 1 0 1 0 ...
 $ nn01Cnt          : num  1 2 0 2 3 4 1 4 1 4 ...
 $ ag01Cnt          : num  0 0 0 0 1 1 0 0 1 2 ...
 $ ag02Cnt          : num  0 0 0 0 1 0 0 0 1 0 ...
 $ vb01Prc          : num  0 0 0 0.0741 0 ...
 $ nn01Prc          : num  0.0909 0.2857 0 0.0741 0.1 ...
 $ ag01Prc          : num  0 0 0 0 0.0333 ...
 $ ag02Prc          : num  0 0 0 0 0.0333 ...
 $ ID               : chr  "919714354467045376" "919714354479722496" "919714354500653056" "919714354542600192" ...
 $ tweet_url        : chr  "/IamJonesy3/status/919714354467045376" "/CCmee_/status/919714354479722496" "/SarahJaneVlogs/status/919714354500653057" "/queentyreen/status/919714354542600195" ...
 $ user_id          : chr  "588400632" "242582835" "29298715" "3290716455" ...
 $ username     

In [ ]:
summary(df2)

In [ ]:
df2 %>%
    group_by(CAMP,TYPE) %>%
    summarise(n = n(), .groups = "drop") 

# Model definition

| variable/factor type        | variable name | symbol |
|:----------------------------|:--------------|--------|
| observation unit (tweet_id) | ID            | u1     |
| outcome (dependent)         | MEAS_VALUE    | d1     |
| between tweet               | CAMP          | b1     |
| between tweet               | TYPE          | b2     |
| within tweet                | MEAS_LEVEL    | w1     |

## Formula 0

In [ ]:
# by-ID random intercepts and by-ID random slopes for MEAS_LEVEL
FORMULA <- MEAS_VALUE~CAMP*TYPE*MEAS_LEVEL+(MEAS_LEVEL||ID)

In [ ]:
# by-ID random intercepts
FORMULA <- MEAS_VALUE~CAMP*TYPE*MEAS_LEVEL+(1|ID) 

In [ ]:
CONTROL <- lmerControl(optCtrl=list(maxfun=1e7))

In [ ]:
CONTROL <- lmerControl(optCtrl=list(maxfun=1e6))

In [ ]:
METHOD <- "LRT" # calculates p-vals via the likelihood ratio tests (use if random factors have > 50 levels)

In [ ]:
METHOD <- "KR"  # Kenward-Roger approximation for degrees of freedom

In [ ]:
METHOD <- "S"   # Satterthwaite approximation for degrees of freedom

In [ ]:
log0 <- logging0(ifName,infoStr="_AFEX_MIXED_formula0")
m0.0 <- mixed(
    formula   = FORMULA,
    data      = df3,
    control   = CONTROL,
    method    = METHOD,
    expand_re = TRUE
)
save(m0.0,file=log0$ofName)
log1 <- logging1(log0$lfName,log0$t0)

In [ ]:
m0.0

## Formula 1

In [ ]:
# by-ID random intercepts and by-ID random slopes for MEAS_LEVEL
FORMULA <- MEAS_VALUE~CAMP*TYPE*MEAS_LEVEL+(MEAS_LEVEL||ID)


# TODO: FIXME: CHECK THIS OUT !!!
CONTROL <- lmerControl(optCtrl=list(maxfun=1e9))

In [ ]:
log0 <- logging0(ifName,infoStr="_AFEX_MIXED_formula1")
m0.1 <- mixed(
    formula   = FORMULA,
    data      = df3,
    control   = CONTROL,
    method    = METHOD,
    expand_re = TRUE
)
save(m0.1,file=log0$ofName)
log1 <- logging1(log0$lfName,log0$t0)

In [ ]:
m0.1

# Quick look at models

In [ ]:
m0.0

In [ ]:
m0.1

## Select model for further inspection

In [ ]:
m0 <- m0.1

In [ ]:
m0 <- m0.0

In [ ]:
summary(m0)$varcor

In [ ]:
m3.0 <- emmeans(m0, c("CAMP", "TYPE", "MEAS_LEVEL"))
m3.0

In [ ]:
pairs(m3.0)

In [ ]:
summary(as.glht(pairs(m3.0)), test=adjusted("free"))

In [ ]:
m2.0 <- emmeans(m0,c("CAMP","TYPE"))
m2.0
pairs(m2.0)
summary(as.glht(pairs(m2.0)),test=adjusted("free"))


# Helpers

In [ ]:
?mixed